In [72]:
import pandas as pd
from FlightRadar24.api import FlightRadar24API as fapi
import pycountry_convert as pc
import time
from geopy import distance
import re

In [2]:
fr_api = fapi()

In [3]:
airports = fr_api.get_airports()
airports_df = pd.DataFrame(data=airports)

In [4]:
airlines = fr_api.get_airlines()
airlines_df = pd.DataFrame(data=airlines)

In [17]:
def flight_details(flight):
    try:
        flight.set_flight_details(fr_api.get_flight_details(flight.id))
    except AttributeError:
        _

In [10]:
flights = fr_api.get_flights()
for flight in flights :
    flight_details(flight)
    time.sleep(0.2)
    # We add the sleep in order to not overload the flightRadar24 API

In [23]:
flights[45].__dict__

{'id': '2d3d4512',
 'icao_24bit': '71C081',
 'latitude': 44.9703,
 'longitude': 24.8336,
 'heading': 293,
 'altitude': 38000,
 'ground_speed': 471,
 'squawk': 'N/A',
 'aircraft_code': 'B789',
 'registration': 'HL8081',
 'time': 1661697704,
 'origin_airport_iata': 'ICN',
 'destination_airport_iata': 'AMS',
 'number': 'KE925',
 'airline_iata': 'KE',
 'on_ground': 0,
 'vertical_speed': -64,
 'callsign': 'KAL925',
 'airline_icao': 'KAL',
 'aircraft_age': 'N/A',
 'aircraft_country_id': 207,
 'aircraft_history': [{'identification': {'id': '2d3ac666',
    'number': {'default': 'KE486'}},
   'airport': {'origin': {'name': 'Da Nang International Airport',
     'code': {'iata': 'DAD', 'icao': 'VVDN'},
     'position': {'latitude': 16.043909,
      'longitude': 108.199303,
      'altitude': 33,
      'country': {'id': None, 'name': 'Vietnam', 'code': 'VNM'},
      'region': {'city': 'Da Nang'}},
     'timezone': {'name': 'Asia/Ho_Chi_Minh',
      'offset': 25200,
      'offsetHours': '7:00',
    

In [135]:
wanted_keys = ['id', 'aircraft_code', 'aircraft_model', 'airline_name', 'destination_airport_country_name',
               'destination_airport_latitude', 'destination_airport_longitude', 'destination_airport_icao',
               'origin_airport_country_name', 'origin_airport_latitude', 'origin_airport_longitude',
               'origin_airport_icao', 'time_details', 'status_text', 'airline_icao', 'aircraft_country_id']
final_flights = [{k: v for k, v in flight.__dict__.items() if k in wanted_keys} for flight in flights]

In [136]:
flights_df = pd.DataFrame(data=final_flights)

In [137]:
flights_df.head(10)

,id,aircraft_code,airline_icao,aircraft_country_id,aircraft_model,airline_name,destination_airport_country_name,destination_airport_latitude,destination_airport_longitude,destination_airport_icao,origin_airport_country_name,origin_airport_latitude,origin_airport_longitude,origin_airport_icao,status_text,time_details
0,2d378c5c,GLID,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,Scheduled,"{'scheduled': {'departure': None, 'arrival': N..."
1,2d386f15,BALL,N/A,236,High Altitude Balloon,Private owner,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,Scheduled,"{'scheduled': {'departure': 0, 'arrival': 0}, ..."
2,2d3870a8,BALL,N/A,236,Balloon,Private owner,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,Scheduled,"{'scheduled': {'departure': 0, 'arrival': 0}, ..."
3,2d38793e,GRND,N/A,55,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,Scheduled,"{'scheduled': {'departure': None, 'arrival': N..."
4,2d38798e,GRND,N/A,0,Ground Vehicle,EVN Airport,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,Scheduled,"{'scheduled': {'departure': None, 'arrival': N..."
5,2d387a5f,GLID,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,Scheduled,"{'scheduled': {'departure': None, 'arrival': N..."
6,2d387c8c,GLID,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,Scheduled,"{'scheduled': {'departure': None, 'arrival': N..."
7,2d387fcc,GLID,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,Scheduled,"{'scheduled': {'departure': None, 'arrival': N..."
8,2d3a947a,GRND,N/A,182,Airport Ground Vehicle,DME Airport,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,Scheduled,"{'scheduled': {'departure': None, 'arrival': N..."
9,2d3ac6d3,GLID,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,Scheduled,"{'scheduled': {'departure': None, 'arrival': N..."


In [138]:
official_flights = flights_df.loc[flights_df['airline_icao'] != 'N/A']

In [139]:
official_flights.head()

,id,aircraft_code,airline_icao,aircraft_country_id,aircraft_model,airline_name,destination_airport_country_name,destination_airport_latitude,destination_airport_longitude,destination_airport_icao,origin_airport_country_name,origin_airport_latitude,origin_airport_longitude,origin_airport_icao,status_text,time_details
12,2d3c2b17,B763,FDX,236,Boeing 767-3S2F,FedEx,N/A,N/A,N/A,N/A,United States,35.042412,-89.976601,KMEM,Scheduled,"{'scheduled': {'departure': 0, 'arrival': 0}, ..."
14,2d3c8b69,A321,ACA,40,Airbus A321-211,Air Canada,N/A,N/A,N/A,N/A,Canada,46.362202,-72.675591,CYRQ,Unknown,"{'scheduled': {'departure': 0, 'arrival': 0}, ..."
17,2d3cfb97,BCS3,ACA,40,Airbus A220-300,Air Canada,United States,40.78838,-111.976997,KSLC,Canada,43.680634,-79.627007,CYYZ,Landed 20:50,"{'scheduled': {'departure': 1661637300, 'arriv..."
18,2d3d0b38,B77L,QTR,179,Boeing 777-2DZ(LR),Qatar Airways,United States,32.89682,-97.037903,KDFW,Qatar,25.272524,51.608604,OTHH,Landed 09:38,"{'scheduled': {'departure': 1661640000, 'arriv..."
19,2d3d0eda,B77L,UAE,234,Boeing 777-21H(LR),Emirates,United States,32.89682,-97.037903,KDFW,United Arab Emirates,25.252769,55.364441,OMDB,Delayed 09:43,"{'scheduled': {'departure': 1661640600, 'arriv..."


### For future questions, we will add the continents names

In [140]:
unknown_to_continent = {
    "Cote D'ivoire (Ivory Coast)" : "AF",
    "Myanmar (Burma)": "AS",
    "Antigua And Barbuda": "NA",
    "Saint Helena": "AF",
    "Bosnia And Herzegovina": "EU",
    "Saint Kitts And Nevis": "NA",
    "Democratic Republic Of The Congo": "AF",
    "Saint Vincent And The Grenadines": "AN",
    "Antarctica": "ANT",
    "Timor-Leste (East Timor)": "AS",
    "Wallis And Futuna": "OC",
    "Turks And Caicos Islands": "NA",
    "Trinidad And Tobago": "SA",
    "Kosovo": "EU",
    "Reunion": "AF",
    "Saint Pierre And Miquelon": "NA",
    "Sao Tome And Principe": "AF",
    "Virgin Islands, Us": "NA",
    "United States Minor Outlying Islands": "NA",
    "Curacao": "SA"
}

def continents(country):
    try:
        return pc.country_alpha2_to_continent_code(pc.country_name_to_country_alpha2(country))
    except KeyError:
        if country in unknown_to_continent.keys():
            return unknown_to_continent[country]
        else:
            return 'Unknown'


In [141]:
official_flights['origin_continent'] = official_flights['origin_airport_country_name'].apply(lambda x: continents(str(x)))
official_flights['destination_continent'] = official_flights['destination_airport_country_name'].apply(lambda x: continents(str(x)))

/tmp/ipykernel_646128/2613344710.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  official_flights['origin_continent'] = official_flights['origin_airport_country_name'].apply(lambda x: continents(str(x)))
/tmp/ipykernel_646128/2613344710.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  official_flights['destination_continent'] = official_flights['destination_airport_country_name'].apply(lambda x: continents(str(x)))


### We will also add a way to know if a flight is regional or not

In [142]:
official_flights['regional'] = official_flights['origin_continent'] == official_flights['destination_continent']

/tmp/ipykernel_646128/2798312429.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  official_flights['regional'] = official_flights['origin_continent'] == official_flights['destination_continent']


In [143]:
official_flights.head()

,id,aircraft_code,airline_icao,aircraft_country_id,aircraft_model,airline_name,destination_airport_country_name,destination_airport_latitude,destination_airport_longitude,destination_airport_icao,origin_airport_country_name,origin_airport_latitude,origin_airport_longitude,origin_airport_icao,status_text,time_details,origin_continent,destination_continent,regional
12,2d3c2b17,B763,FDX,236,Boeing 767-3S2F,FedEx,N/A,N/A,N/A,N/A,United States,35.042412,-89.976601,KMEM,Scheduled,"{'scheduled': {'departure': 0, 'arrival': 0}, ...",NA,Unknown,False
14,2d3c8b69,A321,ACA,40,Airbus A321-211,Air Canada,N/A,N/A,N/A,N/A,Canada,46.362202,-72.675591,CYRQ,Unknown,"{'scheduled': {'departure': 0, 'arrival': 0}, ...",NA,Unknown,False
17,2d3cfb97,BCS3,ACA,40,Airbus A220-300,Air Canada,United States,40.78838,-111.976997,KSLC,Canada,43.680634,-79.627007,CYYZ,Landed 20:50,"{'scheduled': {'departure': 1661637300, 'arriv...",NA,NA,True
18,2d3d0b38,B77L,QTR,179,Boeing 777-2DZ(LR),Qatar Airways,United States,32.89682,-97.037903,KDFW,Qatar,25.272524,51.608604,OTHH,Landed 09:38,"{'scheduled': {'departure': 1661640000, 'arriv...",AS,NA,False
19,2d3d0eda,B77L,UAE,234,Boeing 777-21H(LR),Emirates,United States,32.89682,-97.037903,KDFW,United Arab Emirates,25.252769,55.364441,OMDB,Delayed 09:43,"{'scheduled': {'departure': 1661640600, 'arriv...",AS,NA,False


### We will also add the distances

In [144]:
def calculate_distances(x_lat, x_lon, y_lat, y_lon):
    try:
        return distance.distance((x_lat, x_lon), (y_lat, y_lon)).km
    except Exception:
        return 0

In [145]:
official_flights['flight_distance'] = official_flights.apply(lambda x : calculate_distances(
    x['origin_airport_latitude'],
    x['origin_airport_longitude'],
    x['destination_airport_latitude'],
    x['destination_airport_longitude']), axis=1)

/tmp/ipykernel_646128/1736364439.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  official_flights['flight_distance'] = official_flights.apply(lambda x : calculate_distances(


In [146]:
official_flights.head()

,id,aircraft_code,airline_icao,aircraft_country_id,aircraft_model,airline_name,destination_airport_country_name,destination_airport_latitude,destination_airport_longitude,destination_airport_icao,origin_airport_country_name,origin_airport_latitude,origin_airport_longitude,origin_airport_icao,status_text,time_details,origin_continent,destination_continent,regional,flight_distance
12,2d3c2b17,B763,FDX,236,Boeing 767-3S2F,FedEx,N/A,N/A,N/A,N/A,United States,35.042412,-89.976601,KMEM,Scheduled,"{'scheduled': {'departure': 0, 'arrival': 0}, ...",NA,Unknown,False,0.000000
14,2d3c8b69,A321,ACA,40,Airbus A321-211,Air Canada,N/A,N/A,N/A,N/A,Canada,46.362202,-72.675591,CYRQ,Unknown,"{'scheduled': {'departure': 0, 'arrival': 0}, ...",NA,Unknown,False,0.000000
17,2d3cfb97,BCS3,ACA,40,Airbus A220-300,Air Canada,United States,40.78838,-111.976997,KSLC,Canada,43.680634,-79.627007,CYYZ,Landed 20:50,"{'scheduled': {'departure': 1661637300, 'arriv...",NA,NA,True,2672.466803
18,2d3d0b38,B77L,QTR,179,Boeing 777-2DZ(LR),Qatar Airways,United States,32.89682,-97.037903,KDFW,Qatar,25.272524,51.608604,OTHH,Landed 09:38,"{'scheduled': {'departure': 1661640000, 'arriv...",AS,NA,False,12765.001590
19,2d3d0eda,B77L,UAE,234,Boeing 777-21H(LR),Emirates,United States,32.89682,-97.037903,KDFW,United Arab Emirates,25.252769,55.364441,OMDB,Delayed 09:43,"{'scheduled': {'departure': 1661640600, 'arriv...",AS,NA,False,12939.579079


### I want to get rid of the hour in the status column

In [147]:
official_flights['status_text'] = official_flights['status_text'].apply(lambda x : re.sub('[^a-zA-Z]+', '', str(x)))

/tmp/ipykernel_646128/2771319458.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  official_flights['status_text'] = official_flights['status_text'].apply(lambda x : re.sub('[^a-zA-Z]+', '', str(x)))


In [148]:
official_flights.head()

,id,aircraft_code,airline_icao,aircraft_country_id,aircraft_model,airline_name,destination_airport_country_name,destination_airport_latitude,destination_airport_longitude,destination_airport_icao,origin_airport_country_name,origin_airport_latitude,origin_airport_longitude,origin_airport_icao,status_text,time_details,origin_continent,destination_continent,regional,flight_distance
12,2d3c2b17,B763,FDX,236,Boeing 767-3S2F,FedEx,N/A,N/A,N/A,N/A,United States,35.042412,-89.976601,KMEM,Scheduled,"{'scheduled': {'departure': 0, 'arrival': 0}, ...",NA,Unknown,False,0.000000
14,2d3c8b69,A321,ACA,40,Airbus A321-211,Air Canada,N/A,N/A,N/A,N/A,Canada,46.362202,-72.675591,CYRQ,Unknown,"{'scheduled': {'departure': 0, 'arrival': 0}, ...",NA,Unknown,False,0.000000
17,2d3cfb97,BCS3,ACA,40,Airbus A220-300,Air Canada,United States,40.78838,-111.976997,KSLC,Canada,43.680634,-79.627007,CYYZ,Landed,"{'scheduled': {'departure': 1661637300, 'arriv...",NA,NA,True,2672.466803
18,2d3d0b38,B77L,QTR,179,Boeing 777-2DZ(LR),Qatar Airways,United States,32.89682,-97.037903,KDFW,Qatar,25.272524,51.608604,OTHH,Landed,"{'scheduled': {'departure': 1661640000, 'arriv...",AS,NA,False,12765.001590
19,2d3d0eda,B77L,UAE,234,Boeing 777-21H(LR),Emirates,United States,32.89682,-97.037903,KDFW,United Arab Emirates,25.252769,55.364441,OMDB,Delayed,"{'scheduled': {'departure': 1661640600, 'arriv...",AS,NA,False,12939.579079


In [149]:
official_flights.groupby('status_text')['id'].count()

status_text
Delayed          143
DivertedtoFCO      1
Estimated        674
Landed            66
Scheduled         23
Unknown            9
nan                3
Name: id, dtype: int64

In [150]:
official_flights.shape

(919, 20)

### Now that the data seems to be good, we can answer

## - Q1: What is the company with the most active flights in the world ?


### Due to lack of definition, I will consider an active flight as a Flight having an airline associated, so the Flights in the 'official_flights' DataFrame

In [151]:
official_flights.columns

Index(['id', 'aircraft_code', 'airline_icao', 'aircraft_country_id',
       'aircraft_model', 'airline_name', 'destination_airport_country_name',
       'destination_airport_latitude', 'destination_airport_longitude',
       'destination_airport_icao', 'origin_airport_country_name',
       'origin_airport_latitude', 'origin_airport_longitude',
       'origin_airport_icao', 'status_text', 'time_details',
       'origin_continent', 'destination_continent', 'regional',
       'flight_distance'],
      dtype='object')

In [152]:
official_flights.groupby(['airline_name'], as_index=False)['id'].count().sort_values('id', ascending=False).iloc[0]

airline_name    Delta Air Lines
id                           53
Name: 67, dtype: object

In [153]:
official_flights.groupby(['airline_icao'], as_index=False)['id'].count().sort_values('id', ascending=False).iloc[0]

airline_icao    UAL
id               57
Name: 143, dtype: object

### Counting by airline name, Delta Airline has the most active flights. However, counting by airline icao, United Airlines has the most active flights

## - Q2: By continent, what are the companies with the most regional active flights (airports of Origin & Destination within the same continent) ?


In [154]:
most_regionnal_flights = official_flights.groupby(['origin_continent', 'airline_icao', 'airline_name'], as_index=False)['regional'].sum().sort_values('regional', ascending=False)
most_regionnal_flights.drop_duplicates(subset=['origin_continent'])

,origin_continent,airline_icao,airline_name,regional
255,NA,JBU,JetBlue Airways,18
75,AS,LNI,Lion Air,6
189,EU,SBI,S7 Airlines,4
299,SA,AVA,Avianca,2
315,Unknown,SHW,KN Helicopters,1
289,OC,QFA,Qantas,1
4,AF,ETH,Ethiopian Airlines,1


### The companies having the most regional active flights can be found in the 'airline_name' column

## - Q3: World-wide, Which active flight has the longest route ?


In [155]:
official_flights.iloc[official_flights['flight_distance'].idxmax()]

id                                                                           2d3d7ab5
aircraft_code                                                                    B789
airline_icao                                                                      ANZ
aircraft_country_id                                                               157
aircraft_model                                                Boeing 787-9 Dreamliner
airline_name                                                          Air New Zealand
destination_airport_country_name                                          New Zealand
destination_airport_latitude                                               -37.007999
destination_airport_longitude                                              174.791595
destination_airport_icao                                                         NZAA
origin_airport_country_name                                             United States
origin_airport_latitude                               

### This flight between Oceania and North America has the longest route, with 11 933km.

## - Q4: By continent, what is the average route distance ? (flight localization by airport of origin)


In [156]:
official_flights.groupby(['origin_continent'])['flight_distance'].mean()

origin_continent
AF         6265.529929
AS         8405.238321
EU         6692.933365
NA         8156.040380
OC         7484.106334
SA         6957.998859
Unknown       0.000000
Name: flight_distance, dtype: float64

### Here are the average flight distances for each continent

## - Q5.1: Which leading airplane manufacturer has the most active flights in the world ?


In [157]:
official_flights.groupby(['aircraft_code', 'aircraft_model'], as_index=False)['id'].count().sort_values('id', ascending=False).iloc[:10]

,aircraft_code,aircraft_model,id
167,B789,Boeing 787-9 Dreamliner,130
30,A359,Airbus A350-941,64
165,B788,Boeing 787-8 Dreamliner,56
25,A333,Airbus A330-343,39
115,B77L,Boeing 777-F,21
31,A35K,Airbus A350-1041,21
27,A339,Airbus A330-941,20
157,B77W,Boeing 777-3DZ(ER),16
12,A321,Airbus A321-231,16
106,B772,Boeing 777-223(ER),15


### According to the top ten most seen aircrafts, we see that Boeing is the airplane manufacturer for 238 active flight, whereas Airbus is for 161. So I would say that Boeing has the most.

## - Q5.2: By continent, what is the most frequent airplane model ? (airplane localization by airport of origin)


In [158]:
aircrafts_codes_by_continent = official_flights.groupby(['origin_continent', 'aircraft_code'], as_index=False)['id'].count().sort_values('id', ascending=False)
aircrafts_codes_by_continent.drop_duplicates(subset=['origin_continent'])

,origin_continent,aircraft_code,id
28,AS,B77W,59
42,EU,A333,44
96,NA,B77W,34
117,OC,B789,11
7,AF,B788,6
118,SA,A20N,3
133,Unknown,B77W,1


In [159]:
aircrafts_models_by_continent = official_flights.groupby(['origin_continent', 'aircraft_model'], as_index=False)['id'].count().sort_values('id', ascending=False)
aircrafts_models_by_continent.drop_duplicates(subset=['origin_continent'])

,origin_continent,aircraft_model,id
88,AS,Boeing 787-9 Dreamliner,51
184,EU,Boeing 787-9 Dreamliner,41
279,NA,Boeing 787-9 Dreamliner,23
301,OC,Boeing 787-9 Dreamliner,10
12,AF,Boeing 787-8 Dreamliner,6
315,SA,Boeing 787-9 Dreamliner,3
318,Unknown,Britten-Norman Islander,1


### As one aircraft code can be used for multiple aircraft models, I answered with both. It is interesting to see the big difference, as the models is always the boeing, but when looking at the codes, we see some differences.

## - Q6: By company registration country, what are the tops 3 airplanes model flying ?


#### There is an aircraft country id, and I will assume that this is the company registration country. Sadly, I found no way to associate this ID to a country name, neither in the flightRadar24API code, nor in a python library.

In [160]:
official_flights.head()

,id,aircraft_code,airline_icao,aircraft_country_id,aircraft_model,airline_name,destination_airport_country_name,destination_airport_latitude,destination_airport_longitude,destination_airport_icao,origin_airport_country_name,origin_airport_latitude,origin_airport_longitude,origin_airport_icao,status_text,time_details,origin_continent,destination_continent,regional,flight_distance
12,2d3c2b17,B763,FDX,236,Boeing 767-3S2F,FedEx,N/A,N/A,N/A,N/A,United States,35.042412,-89.976601,KMEM,Scheduled,"{'scheduled': {'departure': 0, 'arrival': 0}, ...",NA,Unknown,False,0.000000
14,2d3c8b69,A321,ACA,40,Airbus A321-211,Air Canada,N/A,N/A,N/A,N/A,Canada,46.362202,-72.675591,CYRQ,Unknown,"{'scheduled': {'departure': 0, 'arrival': 0}, ...",NA,Unknown,False,0.000000
17,2d3cfb97,BCS3,ACA,40,Airbus A220-300,Air Canada,United States,40.78838,-111.976997,KSLC,Canada,43.680634,-79.627007,CYYZ,Landed,"{'scheduled': {'departure': 1661637300, 'arriv...",NA,NA,True,2672.466803
18,2d3d0b38,B77L,QTR,179,Boeing 777-2DZ(LR),Qatar Airways,United States,32.89682,-97.037903,KDFW,Qatar,25.272524,51.608604,OTHH,Landed,"{'scheduled': {'departure': 1661640000, 'arriv...",AS,NA,False,12765.001590
19,2d3d0eda,B77L,UAE,234,Boeing 777-21H(LR),Emirates,United States,32.89682,-97.037903,KDFW,United Arab Emirates,25.252769,55.364441,OMDB,Delayed,"{'scheduled': {'departure': 1661640600, 'arriv...",AS,NA,False,12939.579079


In [185]:
aircraft_by_countries = official_flights.groupby(['aircraft_country_id', 'aircraft_model'], as_index=False)['id'].count().sort_values(['aircraft_country_id', 'id'], ascending=[True,False])

In [195]:
aircraft_by_countries.groupby('aircraft_country_id').nth([0,1,2]).head(45)

,aircraft_model,id
aircraft_country_id,,
0,Airbus A350-941,1
0,Boeing 767-304(ER)(BCF),1
0,Boeing 787-9 Dreamliner,1
4,Airbus A330-202,1
11,Airbus A330-202,1
11,Boeing 737 MAX 8,1
14,Boeing 787-9 Dreamliner,8
14,Airbus A330-202,2
14,Boeing 787-8 Dreamliner,2


#### The DataFrame contains the 3 most popular aircraft models by country id.

## - Q7.1: By continent, what airport is the most popular destination ?


In [200]:
flights_with_destination_airport_name = pd.merge(official_flights, airports_df[['name', 'icao']], right_on='icao', left_on='destination_airport_icao')

In [201]:
flights_with_destination_airport_name.head()

,id,aircraft_code,airline_icao,aircraft_country_id,aircraft_model,airline_name,destination_airport_country_name,destination_airport_latitude,destination_airport_longitude,destination_airport_icao,...,origin_airport_longitude,origin_airport_icao,status_text,time_details,origin_continent,destination_continent,regional,flight_distance,name,icao
0,2d3cfb97,BCS3,ACA,40,Airbus A220-300,Air Canada,United States,40.78838,-111.976997,KSLC,...,-79.627007,CYYZ,Landed,"{'scheduled': {'departure': 1661637300, 'arriv...",NA,NA,True,2672.466803,Salt Lake City International Airport,KSLC
1,2d3db668,A339,DAL,236,Airbus A330-941,Delta Air Lines,United States,40.78838,-111.976997,KSLC,...,2.555752,LFPG,Estimated,"{'scheduled': {'departure': 1661674200, 'arriv...",EU,NA,False,8175.419759,Salt Lake City International Airport,KSLC
2,2d3df3ff,A339,DAL,236,Airbus A330-941,Delta Air Lines,United States,40.78838,-111.976997,KSLC,...,4.763889,EHAM,Estimated,"{'scheduled': {'departure': 1661674500, 'arriv...",EU,NA,False,8030.485073,Salt Lake City International Airport,KSLC
3,2d3e1d04,A320,JBU,236,Airbus A320-232,JetBlue Airways,United States,40.78838,-111.976997,KSLC,...,-81.308899,KMCO,Estimated,"{'scheduled': {'departure': 1661688240, 'arriv...",NA,NA,True,3107.277910,Salt Lake City International Airport,KSLC
4,2d3d0b38,B77L,QTR,179,Boeing 777-2DZ(LR),Qatar Airways,United States,32.89682,-97.037903,KDFW,...,51.608604,OTHH,Landed,"{'scheduled': {'departure': 1661640000, 'arriv...",AS,NA,False,12765.001590,Dallas Fort Worth International Airport,KDFW


In [203]:
airports_ranking_by_continent = flights_with_destination_airport_name.groupby(['destination_continent', 'name'], as_index=False)['id'].count().sort_values('id', ascending=False)
airports_ranking_by_continent.drop_duplicates(subset=['destination_continent'])

,destination_continent,name,id
144,NA,New York John F. Kennedy International Airport,46
86,EU,London Heathrow Airport,24
56,AS,Seoul Incheon International Airport,19
178,OC,Sydney Kingsford Smith Airport,10
180,SA,Bogota El Dorado International Airport,7
8,AF,Johannesburg OR Tambo International Airport,4
192,Unknown,Abidjan Port Bouet Airport,1


## - Q7.2: What airport airport has the greatest inbound/outbound flights difference ? (positive or negative)


#### Don't understand the question, are we looking at what origin_arport has the most differences between regional flights and intercontinental ones ? If not, what is an inbound flight ?

#### I will assume that it is the case, and also that an unknown destination counts as an outbound flight

In [267]:
official_flights['in-out'] = official_flights.apply(lambda x : 1 if x['destination_continent'] == x['origin_continent'] else -1, axis=1)

/tmp/ipykernel_646128/1411821337.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  official_flights['in-out'] = official_flights.apply(lambda x : 1 if x['destination_continent'] == x['origin_continent'] else -1, axis=1)


In [269]:
official_flights.columns

Index(['id', 'aircraft_code', 'airline_icao', 'aircraft_country_id',
       'aircraft_model', 'airline_name', 'destination_airport_country_name',
       'destination_airport_latitude', 'destination_airport_longitude',
       'destination_airport_icao', 'origin_airport_country_name',
       'origin_airport_latitude', 'origin_airport_longitude',
       'origin_airport_icao', 'status_text', 'time_details',
       'origin_continent', 'destination_continent', 'regional',
       'flight_distance', 'duration', 'in-out'],
      dtype='object')

In [273]:
in_out_airports = official_flights.groupby('origin_airport_icao', as_index=False)['in-out'].sum()
in_out_airports['in-out'] = in_out_airports['in-out'].apply(lambda x : abs(x))
in_out_airports.sort_values('in-out', ascending=False).iloc[0]

origin_airport_icao    EGLL
in-out                   46
Name: 22, dtype: object

In [275]:
airports_df.loc[airports_df['icao'] == 'EGLL']['name']

2406    London Heathrow Airport
Name: name, dtype: object

#### The London Heathrow Airport seems to be the airport with the most inbound/outbound

## - Q8: By continent, what is the average active flight speed ? (flight localization by airport of origin)


In [204]:
official_flights.columns

Index(['id', 'aircraft_code', 'airline_icao', 'aircraft_country_id',
       'aircraft_model', 'airline_name', 'destination_airport_country_name',
       'destination_airport_latitude', 'destination_airport_longitude',
       'destination_airport_icao', 'origin_airport_country_name',
       'origin_airport_latitude', 'origin_airport_longitude',
       'origin_airport_icao', 'status_text', 'time_details',
       'origin_continent', 'destination_continent', 'regional',
       'flight_distance'],
      dtype='object')

In [257]:
def calculate_duration(time_details):
    try:
        return int(time_details['historical']['flighttime'])
    except TypeError:
        return 0

In [258]:
official_flights['duration'] = official_flights['time_details'].apply(lambda x : calculate_duration(x))

/tmp/ipykernel_646128/1855033763.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  official_flights['duration'] = official_flights['time_details'].apply(lambda x : calculate_duration(x))


In [259]:
official_flights.head(10)

,id,aircraft_code,airline_icao,aircraft_country_id,aircraft_model,airline_name,destination_airport_country_name,destination_airport_latitude,destination_airport_longitude,destination_airport_icao,...,origin_airport_latitude,origin_airport_longitude,origin_airport_icao,status_text,time_details,origin_continent,destination_continent,regional,flight_distance,duration
12,2d3c2b17,B763,FDX,236,Boeing 767-3S2F,FedEx,N/A,N/A,N/A,N/A,...,35.042412,-89.976601,KMEM,Scheduled,"{'scheduled': {'departure': 0, 'arrival': 0}, ...",NA,Unknown,False,0.000000,0
14,2d3c8b69,A321,ACA,40,Airbus A321-211,Air Canada,N/A,N/A,N/A,N/A,...,46.362202,-72.675591,CYRQ,Unknown,"{'scheduled': {'departure': 0, 'arrival': 0}, ...",NA,Unknown,False,0.000000,0
17,2d3cfb97,BCS3,ACA,40,Airbus A220-300,Air Canada,United States,40.78838,-111.976997,KSLC,...,43.680634,-79.627007,CYYZ,Landed,"{'scheduled': {'departure': 1661637300, 'arriv...",NA,NA,True,2672.466803,0
18,2d3d0b38,B77L,QTR,179,Boeing 777-2DZ(LR),Qatar Airways,United States,32.89682,-97.037903,KDFW,...,25.272524,51.608604,OTHH,Landed,"{'scheduled': {'departure': 1661640000, 'arriv...",AS,NA,False,12765.001590,55592
19,2d3d0eda,B77L,UAE,234,Boeing 777-21H(LR),Emirates,United States,32.89682,-97.037903,KDFW,...,25.252769,55.364441,OMDB,Delayed,"{'scheduled': {'departure': 1661640600, 'arriv...",AS,NA,False,12939.579079,0
20,2d3d0f25,A35K,QTR,179,Airbus A350-1041,Qatar Airways,United States,33.94252,-118.406998,KLAX,...,25.272524,51.608604,OTHH,Estimated,"{'scheduled': {'departure': 1661640000, 'arriv...",AS,NA,False,13366.606077,55261
21,2d3d10c0,A359,AAR,207,Airbus A350-941,Asiana Airlines,Germany,50.037796,8.555783,EDDF,...,37.46907,126.4505,RKSI,Landed,"{'scheduled': {'departure': 1661649000, 'arriv...",AS,EU,False,8565.388869,46556
22,2d3d1110,A35K,QTR,179,Airbus A350-1041,Qatar Airways,Qatar,25.272524,51.608604,OTHH,...,37.618969,-122.374001,KSFO,Landed,"{'scheduled': {'departure': 1661641200, 'arriv...",NA,AS,False,13014.295854,53074
23,2d3d2032,A359,DAL,236,Airbus A350-941,Delta Air Lines,South Africa,-26.139099,28.246,FAOR,...,33.636719,-84.428001,KATL,Estimated,"{'scheduled': {'departure': 1661642700, 'arriv...",NA,AF,False,13581.787793,55046
24,2d3d2541,A388,UAE,234,Airbus A380-861,Emirates,United Arab Emirates,25.252769,55.364441,OMDB,...,37.618969,-122.374001,KSFO,Estimated,"{'scheduled': {'departure': 1661645100, 'arriv...",NA,AS,False,13040.680904,53554


In [260]:
official_flights.iloc[0, -6]

{'scheduled': {'departure': 0, 'arrival': 0},
 'real': {'departure': None, 'arrival': None},
 'estimated': {'departure': None, 'arrival': None},
 'other': {'eta': 0, 'updated': 1661619438},
 'historical': None}

#### As I found no data about the flights speed, I wanted to calculate it myself using the distance between 2 airports and the duration of the flight. But, there are two issues with that method. First, the data is not great with many fields missing or equels None, the statistics would be falsified. And most importantly, it doesn't take into account the eventual connection time between two airports.
#### If I still had to do it, I would try to get the time_details['historical']['flighttime'] and see how it goes.

In [261]:
flights_with_tim_and_distance = official_flights.loc[(official_flights['flight_distance'] > 0) & (official_flights['duration'] > 0)]

In [262]:
flights_with_tim_and_distance.head()

,id,aircraft_code,airline_icao,aircraft_country_id,aircraft_model,airline_name,destination_airport_country_name,destination_airport_latitude,destination_airport_longitude,destination_airport_icao,...,origin_airport_latitude,origin_airport_longitude,origin_airport_icao,status_text,time_details,origin_continent,destination_continent,regional,flight_distance,duration
18,2d3d0b38,B77L,QTR,179,Boeing 777-2DZ(LR),Qatar Airways,United States,32.89682,-97.037903,KDFW,...,25.272524,51.608604,OTHH,Landed,"{'scheduled': {'departure': 1661640000, 'arriv...",AS,NA,False,12765.001590,55592
20,2d3d0f25,A35K,QTR,179,Airbus A350-1041,Qatar Airways,United States,33.94252,-118.406998,KLAX,...,25.272524,51.608604,OTHH,Estimated,"{'scheduled': {'departure': 1661640000, 'arriv...",AS,NA,False,13366.606077,55261
21,2d3d10c0,A359,AAR,207,Airbus A350-941,Asiana Airlines,Germany,50.037796,8.555783,EDDF,...,37.46907,126.4505,RKSI,Landed,"{'scheduled': {'departure': 1661649000, 'arriv...",AS,EU,False,8565.388869,46556
22,2d3d1110,A35K,QTR,179,Airbus A350-1041,Qatar Airways,Qatar,25.272524,51.608604,OTHH,...,37.618969,-122.374001,KSFO,Landed,"{'scheduled': {'departure': 1661641200, 'arriv...",NA,AS,False,13014.295854,53074
23,2d3d2032,A359,DAL,236,Airbus A350-941,Delta Air Lines,South Africa,-26.139099,28.246,FAOR,...,33.636719,-84.428001,KATL,Estimated,"{'scheduled': {'departure': 1661642700, 'arriv...",NA,AF,False,13581.787793,55046


#### Reminder that flight_distance is in km and duration is in seconds

In [263]:
flights_with_tim_and_distance['avg_speed'] = flights_with_tim_and_distance.apply(lambda x : 
                                            (x['flight_distance']*1000/x['duration'])*3.6, axis=1)

/tmp/ipykernel_646128/1476919283.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  flights_with_tim_and_distance['avg_speed'] = flights_with_tim_and_distance.apply(lambda x :


In [264]:
flights_with_tim_and_distance.head()

,id,aircraft_code,airline_icao,aircraft_country_id,aircraft_model,airline_name,destination_airport_country_name,destination_airport_latitude,destination_airport_longitude,destination_airport_icao,...,origin_airport_longitude,origin_airport_icao,status_text,time_details,origin_continent,destination_continent,regional,flight_distance,duration,avg_speed
18,2d3d0b38,B77L,QTR,179,Boeing 777-2DZ(LR),Qatar Airways,United States,32.89682,-97.037903,KDFW,...,51.608604,OTHH,Landed,"{'scheduled': {'departure': 1661640000, 'arriv...",AS,NA,False,12765.001590,55592,826.629834
20,2d3d0f25,A35K,QTR,179,Airbus A350-1041,Qatar Airways,United States,33.94252,-118.406998,KLAX,...,51.608604,OTHH,Estimated,"{'scheduled': {'departure': 1661640000, 'arriv...",AS,NA,False,13366.606077,55261,870.772912
21,2d3d10c0,A359,AAR,207,Airbus A350-941,Asiana Airlines,Germany,50.037796,8.555783,EDDF,...,126.4505,RKSI,Landed,"{'scheduled': {'departure': 1661649000, 'arriv...",AS,EU,False,8565.388869,46556,662.329236
22,2d3d1110,A35K,QTR,179,Airbus A350-1041,Qatar Airways,Qatar,25.272524,51.608604,OTHH,...,-122.374001,KSFO,Landed,"{'scheduled': {'departure': 1661641200, 'arriv...",NA,AS,False,13014.295854,53074,882.757378
23,2d3d2032,A359,DAL,236,Airbus A350-941,Delta Air Lines,South Africa,-26.139099,28.246,FAOR,...,-84.428001,KATL,Estimated,"{'scheduled': {'departure': 1661642700, 'arriv...",NA,AF,False,13581.787793,55046,888.246849


In [266]:
flights_with_tim_and_distance.groupby('origin_continent', as_index=False)['avg_speed'].mean()

,origin_continent,avg_speed
0,AF,843.876155
1,AS,818.617325
2,EU,807.311893
3,NA,831.387177
4,OC,832.768784
5,SA,808.884201


#### Looking online, I saw that the average cruising speed of a plane is about 880 km/h, so the results found are a bit under average, but the:y take into account the time spent before flying, and the time spent going up and then down before and during landing. And, our DataFrame is not the best, so I think this little offset in "normal" 